# 📚 Introduction aux Concepts Fondamentaux

## Bienvenue dans le laboratoire d'apprentissage du modèle de Heston !

Ce notebook introduit les concepts mathématiques fondamentaux de la **finance stochastique** (finance avec aléatoire). Nous utiliserons le vocabulaire technique précis de la discipline tout en expliquant chaque terme.

### 🎯 Objectifs
1. Maîtriser les **processus stochastiques** et leur notation
2. Comprendre les **distributions de probabilité** et leur rôle
3. Implémenter des **simulations Monte Carlo**
4. Comprendre la **volatilité** et pourquoi elle est **stochastique** dans Heston

### 📖 Plan du laboratoire
1. **Ce notebook** - Fondations mathématiques et vocabulaire
2. Processus de Wiener (mouvement Brownien)
3. Modèle de Heston et processus CIR
4. Méthode de Monte Carlo et estimateurs
5. Génération de rapports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Variables Aléatoires et Distributions

### 📖 Définition : Variable Aléatoire (Random Variable)

Une **variable aléatoire** $X$ est une fonction qui associe à chaque résultat d'une expérience aléatoire un nombre réel.

**Notation:** $X : \Omega \to \mathbb{R}$

**En pratique:** Le prix d'un actif demain $S_{t+1}$ est une variable aléatoire car on ne connaît pas sa valeur avant que le marché n'ouvre.

### 📖 Définition : Loi de Probabilité (Probability Distribution)

La **loi de probabilité** d'une variable aléatoire $X$ décrit comment les probabilités sont réparties sur les valeurs possibles de $X$.

On note: $X \sim \mathcal{D}$ ("X suit la distribution D")

### 📖 Distribution Normale (Loi Gaussienne)

La **loi normale** $\mathcal{N}(\mu, \sigma^2)$ est caractérisée par:
- $\mu$ : **espérance** (moyenne théorique)
- $\sigma^2$ : **variance** (dispersion)
- $\sigma$ : **écart-type** (volatilité)

**Fonction de densité de probabilité (PDF):**
$$f(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$$

**Notation:** Si $X \sim \mathcal{N}(\mu, \sigma^2)$, alors:
- $\mathbb{E}[X] = \mu$ (espérance)
- $\text{Var}(X) = \sigma^2$ (variance)
- $\mathbb{P}(a < X < b) = \int_a^b f(x) dx$ (probabilité)

In [ ]:
# Échantillonnage d'une loi normale
# "Échantillonner" = tirer des valeurs aléatoires selon la distribution

mu = 0         # Espérance (première moment)
sigma = 1      # Écart-type (racine carrée du second moment centré)
N = 10000      # Taille de l'échantillon

# Génération de N réalisations indépendantes de X ~ N(μ, σ²)
X = np.random.normal(loc=mu, scale=sigma, size=N)

# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogramme (estimation de la densité)
counts, bins, patches = ax1.hist(X, bins=60, density=True, alpha=0.7, 
                                   color='steelblue', edgecolor='black')
ax1.set_title('Fonction de Densité de Probabilité (PDF) Empirique', 
             fontsize=14, fontweight='bold')
ax1.set_xlabel('x')
ax1.set_ylabel('f(x) - Densité')

# PDF théorique
x_range = np.linspace(mu - 4*sigma, mu + 4*sigma, 200)
pdf_theorique = stats.norm.pdf(x_range, loc=mu, scale=sigma)
ax1.plot(x_range, pdf_theorique, 'r-', linewidth=3, 
        label=f'PDF théorique N({mu}, {sigma}²)')
ax1.axvline(mu, color='green', linestyle='--', linewidth=2, 
           label=f'μ = {mu}')
ax1.axvline(mu + sigma, color='orange', linestyle=':', linewidth=2, 
           label=f'μ ± σ')
ax1.axvline(mu - sigma, color='orange', linestyle=':', linewidth=2)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Fonction de Répartition (CDF)
X_sorted = np.sort(X)
cdf_empirique = np.arange(1, len(X_sorted)+1) / len(X_sorted)
cdf_theorique = stats.norm.cdf(x_range, loc=mu, scale=sigma)

ax2.plot(X_sorted, cdf_empirique, linewidth=2, alpha=0.7, 
        label='CDF Empirique')
ax2.plot(x_range, cdf_theorique, 'r-', linewidth=3, 
        label='CDF Théorique')
ax2.set_title('Fonction de Répartition (Cumulative Distribution Function)', 
             fontsize=14, fontweight='bold')
ax2.set_xlabel('x')
ax2.set_ylabel('F(x) = P(X ≤ x)')
ax2.axhline(0.5, color='gray', linestyle='--', alpha=0.5, 
           label='Médiane (P=0.5)')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Estimateurs empiriques (à partir de l'échantillon):")
print(f"   Espérance empirique: E[X] ≈ {np.mean(X):.4f} (théorique: {mu})")
print(f"   Variance empirique:  Var(X) ≈ {np.var(X, ddof=1):.4f} (théorique: {sigma**2})")
print(f"   Écart-type empirique: σ ≈ {np.std(X, ddof=1):.4f} (théorique: {sigma})")
print(f"\n💡 'ddof=1' utilise l'estimateur non-biaisé (correction de Bessel)")

### 🔑 Concepts Clés

| Terme | Symbole | Signification |
|-------|---------|---------------|
| **Espérance** | $\mathbb{E}[X]$ ou $\mu$ | Valeur moyenne théorique |
| **Variance** | $\text{Var}(X)$ ou $\sigma^2$ | Dispersion autour de la moyenne |
| **Écart-type** | $\sigma$ | Racine carrée de la variance (même unité que X) |
| **PDF** | $f(x)$ | Densité de probabilité (dérivée de la CDF) |
| **CDF** | $F(x) = \mathbb{P}(X \leq x)$ | Probabilité cumulée |
| **Quantile** | $Q(p)$ | Valeur telle que $F(Q(p)) = p$ |

## 2. Processus Stochastique Discret

### 📖 Définition : Processus Stochastique

Un **processus stochastique** est une famille de variables aléatoires indexées par le temps:
$$\{S_t\}_{t \geq 0} \text{ ou } (S_t)_{t=0,1,2,...}$$

**Notation:**
- $S_t$ : valeur du processus au temps $t$
- $S_0$ : condition initiale (valeur à $t=0$)
- $\{S_t\}$ : l'ensemble du processus

**En finance:** Le prix d'un actif $S_t$ est un processus stochastique.

### 📖 Définition : Marche Aléatoire (Random Walk)

Une **marche aléatoire** est définie par:
$$S_{t+1} = S_t + \epsilon_{t+1}$$

où $\epsilon_t \sim \mathcal{N}(\mu \Delta t, \sigma^2 \Delta t)$ sont des **innovations** (chocs aléatoires).

**Propriétés:**
- Les $\epsilon_t$ sont **i.i.d.** (indépendants et identiquement distribués)
- $\mathbb{E}[S_t] = S_0 + t\mu\Delta t$ (drift linéaire)
- $\text{Var}(S_t) = t\sigma^2\Delta t$ (variance croît linéairement)

In [ ]:
# Simulation d'une marche aléatoire avec drift
# Modèle: S_{t+1} = S_t * exp(μΔt + σ√Δt * Z_t)  [version log-normale]

# Paramètres du processus
S0 = 100              # Condition initiale
mu = 0.10             # Drift (rendement annuel espéré)
sigma = 0.20          # Volatilité annuelle (écart-type des log-rendements)
T = 1.0               # Horizon temporel (1 an)
N = 252               # Nombre de pas de temps (jours de trading)
dt = T / N            # Incrément temporel
n_paths = 5           # Nombre de trajectoires (réalisations du processus)

# Initialisation
np.random.seed(42)
t = np.linspace(0, T, N+1)
S = np.zeros((N+1, n_paths))
S[0, :] = S0

# Génération des trajectoires par schéma d'Euler
# Version géométrique: dS/S = μdt + σdW (évite les prix négatifs)
for i in range(N):
    Z = np.random.standard_normal(n_paths)  # Innovations N(0,1)
    S[i+1, :] = S[i, :] * np.exp((mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)

# Visualisation
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Trajectoires des prix
for i in range(n_paths):
    ax1.plot(t, S[:, i], linewidth=2, alpha=0.8, label=f'Trajectoire {i+1}')

# Tendance déterministe (drift)
S_drift = S0 * np.exp(mu * t)
ax1.plot(t, S_drift, 'k--', linewidth=2.5, label='Tendance (drift): S₀e^(μt)')
ax1.axhline(y=S0, color='red', linestyle=':', linewidth=2, alpha=0.7, label='S₀')

ax1.set_title('Processus Stochastique: Mouvement Brownien Géométrique (GBM)', 
             fontsize=14, fontweight='bold')
ax1.set_xlabel('Temps t (années)', fontsize=12)
ax1.set_ylabel('Prix S(t)', fontsize=12)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Log-rendements
log_returns = np.diff(np.log(S), axis=0)
for i in range(n_paths):
    ax2.plot(t[1:], log_returns[:, i], linewidth=1, alpha=0.7)

ax2.axhline(y=mu*dt, color='red', linestyle='--', linewidth=2, 
           label=f'Drift théorique: μΔt = {mu*dt:.6f}')
ax2.set_title('Log-Rendements: r_t = log(S_t/S_{t-1})', fontsize=14, fontweight='bold')
ax2.set_xlabel('Temps t (années)', fontsize=12)
ax2.set_ylabel('Log-rendement', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Résultats de la simulation:")
print(f"\n   Prix finaux S(T):")
for i in range(n_paths):
    rendement = (S[-1, i] - S0) / S0 * 100
    print(f"     Trajectoire {i+1}: S(T) = {S[-1, i]:.2f} (rendement: {rendement:+.2f}%)")

print(f"\n   Prix moyen E[S(T)] ≈ {np.mean(S[-1, :]):.2f}")
print(f"   Théorique: S₀e^(μT) = {S0 * np.exp(mu*T):.2f}")
print(f"\n💡 Le GBM garantit S(t) > 0 toujours (propriété essentielle pour les prix)")

## 3. Méthode de Monte Carlo

### 📖 Définition : Simulation Monte Carlo

La **méthode de Monte Carlo** utilise des simulations aléatoires répétées pour estimer des quantités d'intérêt.

**Principe:** Par la loi des grands nombres,
$$\mathbb{E}[g(X)] \approx \frac{1}{M} \sum_{i=1}^M g(X^{(i)})$$

où $X^{(1)}, ..., X^{(M)}$ sont $M$ réalisations indépendantes de $X$.

### 📖 Définition : Quantiles (Percentiles)

Le **quantile d'ordre $\alpha$** est la valeur $q_\alpha$ telle que:
$$\mathbb{P}(X \leq q_\alpha) = \alpha$$

**Exemples:**
- $q_{0.50}$ = médiane (50% des valeurs en dessous)
- $q_{0.05}$ = VaR 95% (Value at Risk)
- $q_{0.95}$ = seuil des 5% les plus élevés

### 📖 Définition : Intervalle de Confiance

Un **intervalle de confiance à 95%** pour $\mathbb{E}[X]$ est:
$$\text{IC}_{95\%} = \left[ \bar{X} - 1.96 \frac{s}{\sqrt{M}}, \bar{X} + 1.96 \frac{s}{\sqrt{M}} \right]$$

où:
- $\bar{X} = \frac{1}{M}\sum X_i$ : moyenne empirique
- $s = \sqrt{\frac{1}{M-1}\sum (X_i - \bar{X})^2}$ : écart-type empirique
- $\frac{s}{\sqrt{M}}$ : erreur standard (standard error)

In [ ]:
# Simulation Monte Carlo à grande échelle
M = 50000  # Nombre de trajectoires (taille Monte Carlo)

print(f"🎲 Simulation Monte Carlo: M = {M:,} trajectoires")
print(f"   Horizon: T = {T} an")
print(f"   Paramètres: S₀={S0}, μ={mu}, σ={sigma}\n")

np.random.seed(123)

# Génération de M trajectoires
S_MC = np.zeros((N+1, M))
S_MC[0, :] = S0

for i in range(N):
    Z = np.random.standard_normal(M)
    S_MC[i+1, :] = S_MC[i, :] * np.exp((mu - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)

# Prix finaux S(T)
S_T = S_MC[-1, :]

# Statistiques
mean_ST = np.mean(S_T)
std_ST = np.std(S_T, ddof=1)
se_ST = std_ST / np.sqrt(M)  # Erreur standard

# Intervalle de confiance pour E[S(T)]
ic_95_lower = mean_ST - 1.96 * se_ST
ic_95_upper = mean_ST + 1.96 * se_ST

# Quantiles
quantiles = {
    'P1': np.percentile(S_T, 1),
    'P5': np.percentile(S_T, 5),
    'P25': np.percentile(S_T, 25),
    'P50': np.percentile(S_T, 50),  # Médiane
    'P75': np.percentile(S_T, 75),
    'P95': np.percentile(S_T, 95),
    'P99': np.percentile(S_T, 99)
}

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Histogramme avec quantiles
ax1 = axes[0, 0]
counts, bins, patches = ax1.hist(S_T, bins=100, density=True, alpha=0.7,
                                  edgecolor='black', color='steelblue')

# Colorer par zones de probabilité
for i, patch in enumerate(patches):
    if bins[i] < quantiles['P5']:
        patch.set_facecolor('red')  # Queue gauche (5%)
    elif bins[i] > quantiles['P95']:
        patch.set_facecolor('green')  # Queue droite (5%)

ax1.axvline(mean_ST, color='black', linestyle='--', linewidth=2.5, 
           label=f'E[S(T)] = {mean_ST:.2f}')
ax1.axvline(quantiles['P50'], color='blue', linestyle='--', linewidth=2, 
           label=f'Médiane = {quantiles["P50"]:.2f}')
ax1.axvline(quantiles['P5'], color='red', linestyle=':', linewidth=2, 
           label=f'VaR 95% = {quantiles["P5"]:.2f}')
ax1.axvline(quantiles['P95'], color='green', linestyle=':', linewidth=2, 
           label=f'P95 = {quantiles["P95"]:.2f}')
ax1.set_title('Distribution de S(T) - Estimation par Monte Carlo', 
             fontsize=13, fontweight='bold')
ax1.set_xlabel('Prix final S(T)')
ax1.set_ylabel('Densité f(x)')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Box plot
ax2 = axes[0, 1]
bp = ax2.boxplot(S_T, vert=True, patch_artist=True, widths=0.5)
bp['boxes'][0].set_facecolor('lightblue')
bp['medians'][0].set_color('red')
bp['medians'][0].set_linewidth(2)
ax2.set_title('Box Plot - Interprétation des Quantiles', fontsize=13, fontweight='bold')
ax2.set_ylabel('Prix S(T)')
ax2.grid(True, alpha=0.3, axis='y')

# Ajouter annotations
ax2.text(1.15, quantiles['P75'], 'Q3 (P75)', fontsize=10, va='center')
ax2.text(1.15, quantiles['P50'], 'Médiane', fontsize=10, va='center')
ax2.text(1.15, quantiles['P25'], 'Q1 (P25)', fontsize=10, va='center')

# Quantiles tabulés
ax3 = axes[1, 0]
ax3.axis('off')
quant_names = list(quantiles.keys())
quant_values = list(quantiles.values())
colors_q = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(quant_names)))

y_pos = np.arange(len(quant_names))
bars = ax3.barh(y_pos, quant_values, color=colors_q, edgecolor='black')
ax3.set_yticks(y_pos)
ax3.set_yticklabels(quant_names)
ax3.set_xlabel('Prix S(T)')
ax3.set_title('Quantiles de la Distribution', fontsize=13, fontweight='bold')
ax3.invert_yaxis()
ax3.grid(True, alpha=0.3, axis='x')
ax3.axis('on')

# Statistiques textuelles
ax4 = axes[1, 1]
ax4.axis('off')
stats_text = f"""
📊 ESTIMATEURS MONTE CARLO (M = {M:,})
{'='*55}

Moments:
  • E[S(T)] = {mean_ST:.2f} ± {1.96*se_ST:.2f}  (IC 95%)
  • Théorique: S₀e^(μT) = {S0 * np.exp(mu*T):.2f}
  • Écart-type: σ[S(T)] = {std_ST:.2f}
  • Erreur standard: SE = {se_ST:.4f}

Quantiles:
  • Min  = {S_T.min():.2f}
  • P5   = {quantiles['P5']:.2f}  (VaR 95%)
  • P25  = {quantiles['P25']:.2f}  (Q1)
  • P50  = {quantiles['P50']:.2f}  (Médiane)
  • P75  = {quantiles['P75']:.2f}  (Q3)
  • P95  = {quantiles['P95']:.2f}
  • Max  = {S_T.max():.2f}

Intervalles de confiance:
  • IC 95% pour E[S(T)]:
    [{ic_95_lower:.2f}, {ic_95_upper:.2f}]
  
  • IC 90% symétrique:
    [{quantiles['P5']:.2f}, {quantiles['P95']:.2f}]
    (90% des trajectoires terminent dans cet intervalle)

Probabilités:
  • P(S(T) > S₀) = {(S_T > S0).mean()*100:.1f}%
  • P(S(T) > {S0*1.2:.0f}) = {(S_T > S0*1.2).mean()*100:.1f}%
  • P(S(T) < {S0*0.8:.0f}) = {(S_T < S0*0.8).mean()*100:.1f}%

💡 Convergence: Erreur ∝ 1/√M
   Pour réduire l'erreur de moitié, il faut 4× plus de sims!
"""
ax4.text(0.05, 0.5, stats_text, fontsize=10, family='monospace',
        verticalalignment='center', transform=ax4.transAxes)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ Simulation Monte Carlo terminée")
print(f"   Estimateur de E[S(T)]: {mean_ST:.2f}")
print(f"   IC 95%: [{ic_95_lower:.2f}, {ic_95_upper:.2f}]")
print(f"   Valeur théorique: {S0 * np.exp(mu*T):.2f}")
print("="*70)

## 4. Volatilité Stochastique - Motivation du Modèle de Heston

### 📖 Problème : Volatilité Constante

Dans le **modèle de Black-Scholes**, la volatilité $\sigma$ est **déterministe** (constante ou fonction déterministe du temps).

$$dS_t = \mu S_t dt + \underbrace{\sigma}_{\text{constante}} S_t dW_t$$

**Limitations empiriques:**
1. **Smile de volatilité** : la volatilité implicite varie avec le strike
2. **Clustering de volatilité** : périodes de haute/basse volatilité persistantes
3. **Queues épaisses** (fat tails) : plus de valeurs extrêmes qu'avec la loi normale
4. **Asymétrie** (skewness) : corrélation négative prix-volatilité

### 📖 Solution : Volatilité Stochastique

Le **modèle de Heston** (1993) introduit une **volatilité stochastique** $v_t$ qui suit son propre processus aléatoire:

$$\begin{cases}
dS_t = \mu S_t dt + \sqrt{v_t} S_t dW^S_t \\
dv_t = \kappa(\theta - v_t) dt + \sigma_v \sqrt{v_t} dW^v_t
\end{cases}$$

avec $\text{Corr}(dW^S_t, dW^v_t) = \rho$

**Avantages:**
- $v_t$ variable → capture le clustering de volatilité
- $\rho < 0$ → asymétrie (effet de levier)
- Processus CIR pour $v_t$ → $v_t > 0$ toujours
- Formule semi-analytique pour options

In [ ]:
# Comparaison: Volatilité Constante vs Stochastique (aperçu)

# Simulation 1: Volatilité constante (Black-Scholes)
np.random.seed(42)
sigma_const = 0.20
S_BS = np.zeros(N+1)
S_BS[0] = S0

for i in range(N):
    Z = np.random.standard_normal()
    S_BS[i+1] = S_BS[i] * np.exp((mu - 0.5*sigma_const**2)*dt + sigma_const*np.sqrt(dt)*Z)

# Simulation 2: Volatilité stochastique (simplifié)
np.random.seed(42)
v = np.zeros(N+1)  # Variance stochastique
v[0] = sigma_const**2
S_SV = np.zeros(N+1)
S_SV[0] = S0

kappa = 2.0    # Vitesse de retour à la moyenne
theta = 0.04   # Variance de long terme
sigma_v = 0.3  # Vol of vol

for i in range(N):
    Z = np.random.standard_normal()
    
    # Processus de variance (CIR simplifié)
    v[i+1] = v[i] + kappa*(theta - v[i])*dt + sigma_v*np.sqrt(max(v[i], 0))*np.sqrt(dt)*Z
    v[i+1] = max(v[i+1], 1e-6)  # Éviter variance négative
    
    # Prix avec volatilité stochastique
    S_SV[i+1] = S_SV[i] * np.exp((mu - 0.5*v[i])*dt + np.sqrt(v[i])*np.sqrt(dt)*Z)

# Visualisation
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Prix
ax1.plot(t, S_BS, linewidth=2.5, label='Black-Scholes (σ constant)', color='blue')
ax1.plot(t, S_SV, linewidth=2.5, label='Volatilité Stochastique', color='red')
ax1.axhline(y=S0, color='black', linestyle=':', linewidth=2, alpha=0.5)
ax1.set_title('Comparaison: Volatilité Constante vs Stochastique', 
             fontsize=14, fontweight='bold')
ax1.set_xlabel('Temps t')
ax1.set_ylabel('Prix S(t)')
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Volatilité
vol_BS = np.full(N+1, sigma_const)
vol_SV = np.sqrt(v)

ax2.plot(t, vol_BS * 100, linewidth=2.5, linestyle='--', 
        label='Black-Scholes: σ(t) = 20% (constant)', color='blue')
ax2.plot(t, vol_SV * 100, linewidth=2.5, 
        label='Heston: σ(t) = √v(t) (stochastique)', color='red')
ax2.fill_between(t, 0, vol_SV * 100, alpha=0.2, color='red')
ax2.axhline(y=np.sqrt(theta) * 100, color='green', linestyle=':', linewidth=2,
           label=f'Volatilité long terme: √θ = {np.sqrt(theta)*100:.1f}%')

ax2.set_title('Évolution de la Volatilité σ(t) = √v(t)', 
             fontsize=14, fontweight='bold')
ax2.set_xlabel('Temps t')
ax2.set_ylabel('Volatilité (%)')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Observations:")
print("   • Black-Scholes: volatilité FIXE → mouvement régulier")
print("   • Heston: volatilité VARIABLE → capture les régimes de marché")
print("   • La volatilité revient vers θ (mean reversion)")
print("   • σᵥ contrôle l'amplitude des fluctuations de volatilité")
print("\n🎯 Prochain notebook: nous étudierons le processus de Wiener (brownien)")
print("   qui est le fondement mathématique de ces modèles!")

## 🎯 Résumé - Vocabulaire Technique Maîtrisé

### Concepts Fondamentaux

| Concept | Définition | Notation |
|---------|-----------|----------|
| **Variable aléatoire** | Fonction qui associe un nombre à chaque résultat aléatoire | $X: \Omega \to \mathbb{R}$ |
| **Distribution** | Répartition des probabilités | $X \sim \mathcal{N}(\mu, \sigma^2)$ |
| **Espérance** | Valeur moyenne théorique | $\mathbb{E}[X]$ ou $\mu$ |
| **Variance** | Dispersion autour de la moyenne | $\text{Var}(X)$ ou $\sigma^2$ |
| **PDF** | Fonction de densité | $f(x)$ |
| **CDF** | Fonction de répartition | $F(x) = \mathbb{P}(X \leq x)$ |
| **Quantile** | Valeur seuil de probabilité | $Q(\alpha)$ tel que $F(Q) = \alpha$ |
| **Processus stochastique** | Famille de v.a. indexées par le temps | $\{S_t\}_{t \geq 0}$ |
| **Innovation** | Choc aléatoire à chaque pas de temps | $\epsilon_t \sim \mathcal{N}(0, \sigma^2 dt)$ |
| **Drift** | Tendance déterministe | $\mu$ dans $dS = \mu S dt$ |
| **Diffusion** | Partie aléatoire | $\sigma$ dans $dS = \sigma S dW$ |
| **Monte Carlo** | Simulation répétée pour estimer | $\mathbb{E}[X] \approx \frac{1}{M}\sum X^{(i)}$ |
| **Erreur standard** | Précision de l'estimateur | $SE = \frac{s}{\sqrt{M}}$ |
| **IC** | Intervalle de confiance | $[\bar{X} - 1.96 \cdot SE, \bar{X} + 1.96 \cdot SE]$ |

### Modèles

- **Black-Scholes (BS):** $dS_t = \mu S_t dt + \sigma S_t dW_t$ (volatilité constante)
- **Heston:** Volatilité stochastique $v_t$ suivant un processus CIR

### 📖 Prochain Notebook

**02 - Processus de Wiener (Mouvement Brownien)**
- Définition rigoureuse du processus de Wiener $W_t$
- Propriétés: martingale, variation quadratique
- Browniens corrélés: décomposition de Cholesky
- Lien avec les équations différentielles stochastiques (EDS)

---

**Continuez vers 02_Mouvement_Brownien.ipynb** 🚀